# Importing the libraries and data

In [ ]:
library('quanteda')
library('quanteda.textplots')
library("quanteda.textmodels")
library("topicmodels")
library("tidyverse")

In [ ]:
inaugural <- corpus_subset(data_corpus_inaugural, Year <= 1826)

In [ ]:
dfm_inaug <- tokens(inaugural, remove_punct = TRUE) %>% 
    dfm() %>%
    dfm_remove(stopwords('english')) %>%
    dfm_trim(min_termfreq = 10)

In [ ]:
dfm_inaug

In [ ]:
textplot_wordcloud(dfm_inaug)

In [ ]:
tokens(inaugural, remove_punct = TRUE) %>% 
    dfm() %>%
    dfm_remove(stopwords('english')) %>%
    dfm_trim(min_termfreq = 10) %>%
    dfm_weight(scheme = 'count')
    # dfm_weight(scheme = 'boolean')

In [ ]:
tokens(inaugural, remove_punct = TRUE) %>% 
    dfm() %>%
    dfm_remove(stopwords('english')) %>%
    dfm_trim(min_termfreq = 10) %>%
    # dfm_weight(scheme = 'count')
    # dfm_weight(scheme = 'boolean')
    # dfm_tfidf()
    dfm_tfidf(scheme_tf = "prop")

In [ ]:
dfm_inaug <- tokens(inaugural, remove_punct = TRUE) %>% 
    dfm() %>%
    dfm_remove(stopwords('english')) %>%
    dfm_wordstem() %>%
    dfm_trim(min_termfreq = 10) %>%
    dfm_weight(scheme = 'count')

# LSA

In [ ]:
inaug_lsa <- textmodel_lsa(dfm_inaug, nd=5)

In [ ]:
inaug_lsa$docs

In [ ]:
inaug_lsa$features

In [ ]:
head(as.data.frame(inaug_lsa$docs))

In [ ]:
inaug_lsa_longer <- rownames_to_column(as.data.frame(inaug_lsa$docs), var='speech') %>%
                    pivot_longer(cols = V1:V5, names_to = "topics")

In [ ]:
inaug_lsa_longer

In [ ]:
ggplot(inaug_lsa_longer, aes(speech,value,color=topics,fill=topics)) + 
geom_bar(stat='identity') + theme(aspect.ratio = 1) 

This reveals one of the cons of LSA -> negative values in the document-topic matrix.  This poses problems for interpretability.

In [ ]:
predict(inaug_lsa, dfm_inaug[1,])$docs_newspace

In [ ]:
inaug_lsa$sk

In [ ]:
head(inaug_lsa$features)

In [ ]:
word_topic_vals <- as.data.frame(inaug_lsa$features) %>%
                    rownames_to_column(var='words') %>%
                    pivot_longer(cols = V1:V5, names_to = "topics")
word_topic_vals

In [ ]:
word_topic_vals_top5 <- word_topic_vals %>%
                        arrange(desc(value)) %>%
                        group_by(topics) %>%
                        slice(1:5)
word_topic_vals_top5

In [ ]:
as.data.frame(word_topic_vals_top5)

In [ ]:
ggplot(data = word_topic_vals_top5, 
       aes(x = nrow(word_topic_vals_top5):1, y = value)) +
geom_point() +
facet_wrap(~ topics, scales = "free") +
coord_flip() + 
scale_x_continuous(breaks = nrow(word_topic_vals_top5):1,
            labels = word_topic_vals_top5$words)

# LDA

In [ ]:
dfm_inaug <- tokens(inaugural, remove_punct = TRUE) %>% 
    dfm() %>%
    dfm_remove(stopwords('english')) %>%
    dfm_wordstem() %>%
    dfm_trim(min_termfreq = 10) %>%
    dfm_weight(scheme = 'count')

In [ ]:
lda5 <- convert(dfm_inaug, to = "topicmodels") %>% 
        LDA(k = 5)

In [ ]:
terms(lda5, 10)

In [ ]:
dfm_inaug <- tokens(inaugural, remove_punct = TRUE) %>% 
    dfm() %>%
    dfm_remove(stopwords('english')) %>%
    dfm_wordstem() %>%
    dfm_trim(min_termfreq = 10) %>%
    dfm_weight(scheme = 'boolean')

In [ ]:
dfm_inaug

In [ ]:
lda5 <- convert(dfm_inaug, to = "topicmodels") %>% 
        LDA(k = 5)

In [ ]:
terms(lda5, 10)

In [ ]:
topics(lda5)

In [ ]:
lda5@beta

In [ ]:
as.data.frame(t(lda5@beta), row.names = lda5@terms)

In [ ]:
as.data.frame(lda5@gamma, row.names = lda5@documents)

In [ ]:
doc_topic_longer <- as.data.frame(lda5@gamma, row.names = lda5@documents) %>%
                    rownames_to_column(var='speech') %>%
                    pivot_longer(cols = V1:V5, names_to = "topic")

In [ ]:
doc_topic_longer

In [ ]:
ggplot(doc_topic_longer, aes(value, speech, color=topic, fill=topic)) + 
geom_bar(stat='identity') + theme(aspect.ratio = 1) 

In [ ]:
lda5_word_topic_vals <- as.data.frame(t(lda5@beta), row.names = lda5@terms)

In [ ]:
word_topic_vals <- lda5_word_topic_vals %>%
                    rownames_to_column(var='words') %>%
                    pivot_longer(cols = V1:V5, names_to = "topics")
word_topic_vals

In [ ]:
word_topic_vals_top15 <- word_topic_vals %>%
                        arrange(desc(value)) %>%
                        group_by(topics) %>%
                        slice(1:15)
word_topic_vals_top15

In [ ]:
ggplot(data = word_topic_vals_top15, 
       aes(x = nrow(word_topic_vals_top15):1, y = value)) +
geom_point() +
facet_wrap(~ topics, scales = "free") +
coord_flip() + 
scale_x_continuous(breaks = nrow(word_topic_vals_top15):1,
            labels = word_topic_vals_top15$words)